In [1]:
!pip install efficientnet_pytorch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from sklearn import metrics

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=f703d1a9e8d2b69aa64b78740c43a0054223f55ed5ec8316d8e9b15e0009e82a
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def seed_everything(seed=42):
    #random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
class Dataset(Dataset):
    def __init__(self, root_images, root_file, mode, fold, transform=None):
        self.root_images = root_images
        self.transform = transform
        self.root_file = root_file

        if mode == 'train':
            self.file = pd.read_csv(self.root_file)[pd.read_csv(self.root_file)['fold'] != fold]
            result    = pd.concat([self.file, self.file[self.file['extent']!=0]])
            self.file = pd.concat([result, self.file[self.file['extent']!=0]]).reset_index(drop=True)
            
        else:
            self.file = pd.read_csv(self.root_file)[pd.read_csv(self.root_file)['fold'] == fold].reset_index(drop=True)
    

    def __len__(self):
        return self.file.shape[0]
    
    def __getitem__(self, index):
        img_path     = os.path.join(self.root_images, self.file['filename'][index])
        image        = np.array(Image.open(img_path).convert('RGB'))
        label        = self.file['extent'][index]

        if self.transform is not None:
            augmentations = self.transform(image=image)
            image         = augmentations['image']

        return image,  torch.tensor(label)

In [4]:
LR    = 1e-4
BS    = 16
NE    = 100
H     = 256+20
W     = 300+20

IMG_Train  = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/train'
IMG_Test   = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/test'

FILE_Train = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/train_folds.csv'
FILE_Test  = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/Test.csv'

In [5]:
train = pd.read_csv(FILE_Train)
test = pd.read_csv(FILE_Test)
SampleSubmission = pd.read_csv('/kaggle/input/cgiar-eyes-on-the-ground-challenge/SampleSubmission.csv')

In [6]:
def get_loaders(image, file, fold, bs, train_transform, val_transform):
    
    train_ds      = Dataset(root_images=image, root_file=file, mode='train', fold=fold, transform=train_transform )
    train_loader  = DataLoader(train_ds, batch_size=bs, shuffle=True)

    val_ds        = Dataset(root_images=image, root_file= file, mode='val', fold=fold, transform=val_transform )
    val_loader    = DataLoader(val_ds, batch_size = bs, shuffle=False)

    return train_loader, val_loader

In [7]:
normalize = A.Normalize(
    mean=[0.5, 0.5, 0.5], 
    std = [0.5, 0.5, 0.5], 
  #  max_pixel_value= 255.0
)

def aug_1():
    train_transform = A.Compose(
        [
            A.Resize(width=W, height=H),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=30),
            A.RandomCrop(height=H-20, width=W-20, p=1),
            normalize,
            ToTensorV2()
        ],
    )

    val_transform = A.Compose(
        [
            A.Resize(width=W, height=H),
            normalize,
            ToTensorV2()
        ],
    )
    return train_transform, val_transform
        
train_transform, val_transform = aug_1()

In [8]:
def check_loader():
    train_loader, val_loader = get_loaders(
            IMG_Train, FILE_Train, 1, BS, train_transform, val_transform
        )
    image, y = next(iter(train_loader))
    print(image.shape, y.shape)

    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(((image[i]+1)/2).permute(1,2,0))
        plt.axis("off")
        name = y[i].item()
        ax.set_title(name)
#check_loader()

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.modelA = EfficientNet.from_pretrained("efficientnet-b5")
        self.fc1 = nn.Linear(1000, 1)

        self.relu = nn.ReLU()
        
    def forward(self, img):
        x = self.modelA(img)
        x = self.fc1(x)
        return self.relu(x)

#x = torch.randn(4,3,224,192)
#m = Net()
#m(x)

In [10]:
def check_acc(loader, model, loss_fn):
    error = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x     = x.to('cuda').to(torch.float32)
            y     = y.to(torch.float32).unsqueeze(1)

            p     =  model(x).cpu()
            error += metrics.mean_squared_error(y, p, squared=False)
            
        print('RMSE : '+ str(error/len(loader)))
    model.train()
    return error/len(loader)

def save_checkpoint(state, filename):
    print('--> Saving Checkpoint')
    torch.save(state, filename)

In [11]:
def train_fn(loader, model, optimizer, loss_fn):
    for x, y in tqdm(loader):
        x     = x.to('cuda').to(torch.float32)
        y     = y.to('cuda').to(torch.float32).unsqueeze(1)

        # Forward
        scores = model(x)
        loss   = loss_fn(scores, y.float())
        # Backward
        model.zero_grad()
        loss.backward()
        optimizer.step()

In [12]:
loss_fn   = nn.MSELoss().to('cuda') 

for fold in range(1):
    print(' --------------------------------------------------- Fold: ' +str(fold))

    model     = Net().to('cuda')
    optimizer = optim.Adam(model.parameters(), lr = LR)

    train_loader, val_loader = get_loaders(
        IMG_Train, FILE_Train, fold, BS, train_transform, val_transform
    )

    l  = 100
    es = 0
    #check_acc(val_loader, model, loss_fn)

    for epoch in range(NE):
        print(' ----------------------------- Epoch: ' +str(epoch))
        
        train_fn(train_loader, model, optimizer, loss_fn)
        loss = check_acc(val_loader, model, loss_fn)
        if loss < l:
            l = loss
            checkpoint = {
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict()
            }
            save_checkpoint(checkpoint, filename = 'baseline_' + str(fold)+ '.pth.tar')
            es = 0
        else:
            es += 1
        
        if es == 3: break

 --------------------------------------------------- Fold: 0


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth
100%|██████████| 117M/117M [00:01<00:00, 80.9MB/s]


Loaded pretrained weights for efficientnet-b5
 ----------------------------- Epoch: 0


100%|██████████| 326/326 [03:13<00:00,  1.69it/s]


RMSE : 11.405298395756564
--> Saving Checkpoint
 ----------------------------- Epoch: 1


...

RMSE : 10.08334189379142
 ----------------------------- Epoch: 9


100%|██████████| 326/326 [02:22<00:00,  2.29it/s]


RMSE : 10.082183516601477
 ----------------------------- Epoch: 10


100%|██████████| 326/326 [02:23<00:00,  2.27it/s]

RMSE : 10.294456463375706
